# **In this notebook i will show how easy to take 0.99475 in Public Score.**


# Import necessary libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

from sklearn.model_selection import StratifiedKFold 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# Load data from csv files

In [2]:
data_train = pd.read_csv("../input/digit-recognizer/train.csv")
X_test = pd.read_csv("../input/digit-recognizer/test.csv")

# Split data and labels and normalize images

In [3]:
X, y = data_train.drop(labels = ["label"],axis = 1)/255.,data_train["label"]
X_test = X_test/255.

# Reshape images to (batch_size x height x width x channels)

In [4]:
X = X.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

# Define a training strategy we reduct learning after 3 epochs without improvements,and stop the train after 30 epochs without improvements

In [5]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3,# How many epochs to wait before reduct lerning rate
                                            verbose=1, 
                                            factor=0.3, 
                                            min_lr=0.00001)
early_stopping = EarlyStopping(
    min_delta=0.000001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# We split the dataset on 7 folds.The folds are made by preserving the percentage of samples for each class.

You can change here:
* n_splits(number to splits,and see how to change the result)
* batch_size(for instance take batch_size  200 or 50)
* or change  probability of an element to be zeroed in Dropout layers
* change patience in early_stopping and learning_rate_reduction

In [6]:
skf = StratifiedKFold(n_splits=3,random_state=42,shuffle=True)
sub = pd.DataFrame(data=None, index=(range(1,28001)), columns=None, dtype=None, copy=False)
for train_index, val_index in skf.split(X, y):
    model = keras.Sequential([
    keras.layers.Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(28*28, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(28*28, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(28*28, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    ])
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    history = model.fit(
    x=X_train, y=y_train, batch_size=100, epochs=250, verbose=1, callbacks=[early_stopping,learning_rate_reduction],
    validation_data=(X_val,y_val), shuffle=True)
    # predict results
    results = model.predict(X_test)
    # select the index with the maximum probability
    results = np.argmax(results,axis = 1)
    results = pd.Series(results,name="Label")
    sub = pd.concat([sub, results],axis=1)


Epoch 1/250
280/280 [==============================] - 2s 7ms/step - loss: 0.4765 - accuracy: 0.8514 - val_loss: 17.9308 - val_accuracy: 0.1116
Epoch 2/250
280/280 [==============================] - 2s 6ms/step - loss: 0.1747 - accuracy: 0.9489 - val_loss: 0.9501 - val_accuracy: 0.8173
Epoch 3/250
280/280 [==============================] - 2s 6ms/step - loss: 0.1266 - accuracy: 0.9618 - val_loss: 0.0823 - val_accuracy: 0.9776
Epoch 4/250
280/280 [==============================] - 2s 6ms/step - loss: 0.1044 - accuracy: 0.9689 - val_loss: 0.0648 - val_accuracy: 0.9839
Epoch 5/250
280/280 [==============================] - 2s 6ms/step - loss: 0.0947 - accuracy: 0.9711 - val_loss: 0.0691 - val_accuracy: 0.9811
Epoch 6/250
280/280 [==============================] - 2s 6ms/step - loss: 0.0867 - accuracy: 0.9735 - val_loss: 0.0523 - val_accuracy: 0.9859
Epoch 7/250
280/280 [==============================] - 2s 6ms/step - loss: 0.0741 - accuracy: 0.9776 - val_loss: 0.0463 - val_accuracy: 0.987

# Making a submission

In [7]:
sub["result"] = sub.mode(dropna=True,axis=1)[0]
result = pd.Series(sub["result"],name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),result],axis = 1)
submission = submission.dropna().astype('int32')
submission.to_csv("mnist_ansamble_of_cnn.csv",index=False)